In [1]:
import json
from os import stat
import numpy as np
import pandas as pd
from datetime import date, timedelta
import time
import urllib.request as ur
import os
import sys

# API calls

1. [Rainfall](https://api.data.gov.sg/v1/environment/rainfall)
2. [Air temperature](https://api.data.gov.sg/v1/environment/air-temperature)
3. [Wind Speed](https://api.data.gov.sg/v1/environment/wind-speed)
4. [Air pollution](https://api.data.gov.sg/v1/environment/psi)

The API is used by adding the suffix, `?date={to be added}` after the links above. This then looks up the data from the API which is returned as a json file.

The `parse_api` function is called to do two things. These are 

1. read the data from the url using the api
2. converting that data from the json into a python object

Objective 1 is done through the `urllib.request` package. First, the correct url is given to a `urllib.request` object. Once the correct url is read, the obj is then read and parsed with a json reader from the python package `json`. The read json, now called `data_obj` is then returned.

In [2]:
def parse_api(url, year):
    url =  url.format(str(year))
    # print(url)
    fileobj = ur.urlopen(url)
    readobj = fileobj.read()
    data_obj = json.loads(readobj)
    return data_obj

The `get_dates` function is called to obtain all the dates between the start and end date using the `datetime` package. The function takes the start and end date as inputs, and returns all dates inclusive of the end date in one day intervals. 

*TO-DO*
- [x] Add in a way to set user defined date ranges

In [3]:
def get_dates(sdate, edate, interval = 1):
    delta = edate - sdate       # as timedelta
    dates = []

    for i in range(0, delta.days + 1, interval):
        day = sdate + timedelta(days=i)
        dates.append(day)
    
    return np.array(dates, dtype = object)

The `get_stations` function takes the read `data_obj` and obtains all metadata from `data_obj` related to the identity and location of the reporting stations. This is then converted into a dataframe, and returned. 

In [4]:
def get_stations(data_obj):
    stations = data_obj["metadata"]["stations"]
    df = pd.DataFrame.from_dict(stations)

    locations = df['location'].to_numpy()
    df['longitude'] = np.zeros(locations.size)
    df['latitude'] = np.zeros(locations.size)

    for i in range(locations.size):
        df.loc[i, 'longitude'] = float(locations[i]['longitude'])
        df.loc[i, 'latitude'] = float(locations[i]['latitude'])

    df = df.drop(['location'], axis = 1)
    return df 

The `get_data_day` function takes in the information of the stations and the `data_obj` and returns a dataframe that represents all rainfall data obtained for that day. Usually data is taken at 5 minute intervals throughout the day from all stations. 

The flow of this function is to 

1. Obtain all the station id's
2. Obtain the dictionary of all rainfall data that day from the json file
3. Extract the time that the reading was taken
4. Extract all rainfall data from the stations throughout the dat 
5. Write them into a dataframe

Return dataframe

In [5]:
def get_data_day(data_obj, stations):
    keys = np.array(stations['id'])
    precipitation_raw = data_obj["items"]
    n = len(precipitation_raw)
    t = np.zeros(n, dtype = object)
    df = pd.DataFrame(np.zeros((n, keys.size)), columns= keys)

    for i, d_obj in enumerate(precipitation_raw):
        t[i] = d_obj['timestamp'].split('+')[0].split('T')[-1]
        list_timestep = d_obj['readings']
        for j in range(len(list_timestep)):
            dict_key = list_timestep[j]['station_id']
            value = list_timestep[j]['value']
            df[dict_key].iloc[i] += value
    
    df['Time'] = t
    df = df.set_index('Time')
    return df

In [6]:
def inputs():
    sdate = sys.argv[1]
    edate = sys.argv[2]
    if len(sys.argv) < 4:
        save_folder = 'raw_data'
    else:
        save_folder = sys.argv[3]
    return sdate, edate, save_folder

*TO-DO*
1. Add command line input system

In [9]:
url = "https://api.data.gov.sg/v1/environment/rainfall?date={0}"
save_folder = 'raw_data'

# sdate, edate, save_folder = inputs()

try:
    os.mkdir('./{0}'.format(save_folder))
except FileExistsError:
    print('Folder exists')

# sdate = date(2022, 1, 1)   # start date
# edate = date.today()   # end date
# dates = get_dates(sdate, edate)

years = np.arange(2018, 2023)
# months = np.arange(6, 9)

dates = []

for i, year in enumerate(years):
    sdate = date(year, 3, 1)
    edate = date(year, 9, 1)

    temp_date = get_dates(sdate, edate, interval = 5)

    for d in temp_date:
        dates.append(d)

# sdate = date(int(*sdate.split("_")))
# edate = date(int(*edate.split("_")))

for i, d in enumerate(dates):
    data_obj = parse_api(url, d)
    print(d)
    if len(data_obj['metadata']['stations']) == 0:
        print('No valid stations on data from', str(d))
        continue
    else:
        stations = get_stations(data_obj)
        precipitation = get_data_day(data_obj, stations)
        stations.to_csv('./{0}/{1}_stations.csv'.format(save_folder, d))
        precipitation.to_csv('./{0}/{1}_precipitation.csv'.format(save_folder, d))

Folder exists
2018-03-01
2018-03-06
2018-03-11
2018-03-16
2018-03-21
2018-03-26
2018-03-31
2018-04-05
2018-04-10
2018-04-15
No valid stations on data from 2018-04-15
2018-04-20
2018-04-25
2018-04-30
2018-05-05
2018-05-10
2018-05-15
2018-05-20
2018-05-25
2018-05-30
2018-06-04
2018-06-09
2018-06-14
2018-06-19
2018-06-24
2018-06-29
2018-07-04
2018-07-09
2018-07-14
2018-07-19
2018-07-24
2018-07-29
2018-08-03
2018-08-08
2018-08-13
2018-08-18
2018-08-23
2018-08-28
2019-03-01
2019-03-06
2019-03-11
2019-03-16
2019-03-21
2019-03-26
2019-03-31
2019-04-05
2019-04-10
2019-04-15
2019-04-20
2019-04-25
2019-04-30
2019-05-05
2019-05-10
2019-05-15
2019-05-20
2019-05-25
2019-05-30
2019-06-04
2019-06-09
2019-06-14
2019-06-19
2019-06-24
2019-06-29
2019-07-04
2019-07-09
2019-07-14
2019-07-19
2019-07-24
No valid stations on data from 2019-07-24
2019-07-29
2019-08-03
2019-08-08
2019-08-13
2019-08-18
2019-08-23
2019-08-28
2020-03-01
2020-03-06
2020-03-11
2020-03-16
2020-03-21
2020-03-26
2020-03-31
2020-04-05


In [ ]:
years = np.arange(2000, 2023)
months = np.arange(6, 9)

In [ ]:


dates